In [242]:
import numpy as np
import torch
from torch.utils.data import DataLoader

def load_128bit_samples(filename):
    with open(filename, 'rb') as f:
        raw = np.frombuffer(f.read(), dtype=np.uint8)

    all_bits = np.unpackbits(raw)
    # Each sample = 128 bits (128 inputs)
    num_samples = all_bits.size // 128
    all_bits = all_bits[:num_samples * 128]

    inputs = all_bits.reshape((num_samples, 128))
    return inputs

In [243]:
def load_labels(filename, n):
    labels = torch.empty(n, 65, dtype=torch.float32)
    with open(filename, 'r') as f:
        for i in range(n):
            labels[i] = torch.tensor(list(map(float, f.readline().split())), dtype=torch.float32)
    return labels


In [244]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs_uint8, policy, value, transform = None):
        self.inputs_uint8 = inputs_uint8
        self.policy = policy
        self.value = value
        self.transform = transform

    def __len__(self):
        return len(self.inputs_uint8)

    def __getitem__(self, idx):
        inputs = torch.tensor(self.inputs_uint8[idx], dtype=torch.uint8).float()
        policy = torch.tensor(self.policy[idx], dtype=torch.float32)
        value = torch.tensor(self.value[idx], dtype=torch.float32)
        if self.transform:
            inputs = self.transform(inputs)

        return inputs, policy, value

In [245]:
inputs = load_128bit_samples("datasets/features.bin")
n = len(inputs)
labels = load_labels("datasets/labels.txt", n)
policies = labels[:, :64]
values = labels[:, 64:]
dataset = Dataset(inputs, policies, values)
training_data, test_data = torch.utils.data.random_split(dataset, [0.8, 0.2])

BATCH_SIZE = 64
LEARNING_RATE = 0.3
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [246]:
from nn import NeuralNetwork, load_model
with open('current_generation.txt', 'r') as f:
    nn_name = f.readline()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = load_model(NeuralNetwork, 'models/model_weights_' + nn_name + '.pth')

Using cpu device


In [247]:
def loss_fn(prediction, target):
    # could play around with the proportions later
    # policy
    loss = torch.nn.functional.cross_entropy(prediction[0], target[0])
    # value
    loss += torch.nn.functional.mse_loss(prediction[1], target[1])
    return loss

optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE, weight_decay = 1e-4)

In [248]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (input, policy, value) in enumerate(dataloader):
        input, policy, value = input.to(device), policy.to(device), value.to(device)
        prediction = model(input)
        # X, y = model(input)
        # print(X)
        # print("------------------------------------")
        # print(y)
        loss = loss_fn(prediction, (policy, value))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (batch + 1) % 2000 == 0:
            loss, current = loss.item(), batch * BATCH_SIZE
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")

In [249]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for input, policy, value in dataloader:
            input, policy, value = input.to(device), policy.to(device), value.to(device)
            prediction = model(input)
            test_loss += loss_fn(prediction, (policy, value)).item()

    test_loss /= num_batches
    # print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")
    print(f"Avg Loss: {test_loss:>8f} \n")
    return test_loss

In [250]:
patience = 2
epochs_without_improvement = 0
best_val_loss = float("inf")
epochs = 5
bestNN = None
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss = test_loop(test_dataloader, model, loss_fn)
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        bestNN = model
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            break  # early stopping

print("Done!")

Epoch 1
-------------------------------
Avg Loss: 4.364673 

Epoch 2
-------------------------------
Avg Loss: 4.327793 

Epoch 3
-------------------------------
Avg Loss: 4.299311 

Epoch 4
-------------------------------
Avg Loss: 4.276863 

Epoch 5
-------------------------------
Avg Loss: 4.265021 

Done!


/tmp/ipykernel_40000/3887164103.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  policy = torch.tensor(self.policy[idx], dtype=torch.float32)
/tmp/ipykernel_40000/3887164103.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(self.value[idx], dtype=torch.float32)
